In [18]:
import random
import pandas as pd
import numpy as np
import

from mesa import Agent
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.space import Grid

from schedule import RandomActivationByBreed
# from agent import Sensor, Gateway, Cloud, Blockchain


In [185]:

class Sensor(Agent):
    """
    An empirical sensor with battery, data store,
    processor and information transceiver.

    Attributes:
        unique_id: Universally unique identifier. Here, a randomly generated id.
            This could be a wallet address or public key, plus private key
        pos: An (x,y) tuple of the connected sensor's spatial position
        sync_freq: The frequency with which the sensor syncs with the cloud.
            If float 0.0 < n < 1.0, this is the probability of syncing on a given tick.
            If int n >= 1, this sensor syncs regularly every n ticks.
        parent_gateway: The superior gateway ID that the sensor reports to.
        battery_life: The amount of battery available. If -999, the sensor
            is considered to be connected to a stable power source.
        accuracy: A float value representing the difference between the observed
            and actual value
        precision: A float value representing the standard deviation of distribution
            of measurements around observed value


        # attributes: The types of sensors connected to the device,
        #     including "temperature", "moisture", "pressure",
        #     "orientation", "light".

        # storage: The amount of data storage available on device in GB
        #     (i.e. SD card size)
        # computer_power: The processor speed in MHz
        # schema: The format the data is collected in, to be used in measuring
        #     interoperability


    """

    def __init__(self, unique_id, pos, sync_freq, parent_gateway, battery_life, accuracy, precision, model):

        super().__init__( unique_id, model)

        self.agent_type = "Sensor"
        self.unique_id = unique_id
        self.pos = pos
        self.sync_freq = sync_freq
        self.parent_gateway = parent_gateway # agent id ...
        self.battery_life = battery_life
        self.accuracy = accuracy
        self.precision = precision

        actual = random.randint(0,100)

        self.records = pd.DataFrame(columns = ['tick', 'actual', 'observed']).set_index('tick')
        self.records.actual = self.records.actual.astype(float)
        self.records.observed = self.records.observed.astype(float)

        
    # def __init__(self, uuid, attributes, battery_life, storage, compute_power, schema, model):
    #     super().__init__(self, uuid, attributes, battery_life, storage, compute_power, schema, model)
    #
    #     self.uuid = uuid
    #     self.attributes = attributes
    #     self.battery_life = battery_life
    #     self.storage = storage
    #     self.compute_power = compute_power
    #     self.schema = schema
    #     self.recordings = []


#     def transmit(self):

    def step(self):
        """

        """

        # Actual condition at sensor position:
        actual = self.model.calc_actual(self.pos)
        print("Actual:", actual)
        
        # Observed condition at sensor position:
        observed = self.observe(actual)
        print("Observed:", observed)

        self.record(actual, observed)
        

    def observe(self, actual):
        observed = np.random.normal(actual + self.accuracy, self.precision)

        return observed
         

    def record(self, actual, observed):
        self.records.loc[self.model.schedule.steps, "actual"] = actual

        # Calculate recorded value based on actual and sensor accuracy and precision valuess
        self.records.loc[self.model.schedule.steps, "observed"] = observed # << inject error here

    def summarize(self, start, end):
        summary = self.records[start:end].describe()
        summary.loc['start_tick'] = start
        summary.loc['end_tick'] = end
        summary.loc['sensor_id'] = self.unique_id
        
        return summary
        
        
    def transmit(self, target_id, data, signed=False):
        
        self.model._agents[target_id].receive(self.unique_id, data)
        
        if signed:
            self.energy -= self.model.signature_cost

   

In [164]:

class Gateway(Agent):
    """
    A computing node positioned between edge Sensors and the
    broader internet.

    Attributes:
        child_sensors: Sensors connected to the gateway, distal on the
            network tree
        storage: The amount of data storage available on device in GB
    """

    def __init__(self, unique_id, child_sensors, storage, parent_cloud, model):
        super().__init__(unique_id, model)

        self.agent_type = "Gateway"
        self.unique_id = unique_id
        self.child_sensors = {}
        self.storage = storage
        self.parent_cloud = parent_cloud

    def step(self):
        pass

    def receive(self, origin_id, data):
        self.child_sensors[origin_id].append(data)



In [21]:
class Cloud(Agent):
    """
    A centralized data storage and computing environment
    """
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

        self.agent_type = "Cloud"
        self.unique_id = unique_id




In [127]:

class CentralizedArchitecture(Model):
    """
    Simulating traditional cloud server web architectures including
    client-server relationship
    """

    def __init__(self, num_clouds, gateways_per_cloud,
                sensors_per_gateway, sensor_accuracy, sensor_precision, height=100, width=100):
        """
        Create a new model with centralized database architectures.

        """
        super().__init__()
        
        self.signature_cost = 0.001

        self.height = height
        self.width = width

        self.schedule = RandomActivationByBreed(self)
        self.grid = Grid(height, width, torus=False)

        self.datacollector = DataCollector(
            {"Temperature": lambda m: self.retrieve_state_updates(m)
            })

        # self.agents_by_breed = defaultdict(dict)

        for cloud in range(0, num_clouds):

            current_cloud_id = self.next_id()
            new_cloud = Cloud(current_cloud_id, self)
            # self.grid._place_agent() # where do the cloud symbols go?
            self.schedule.add(new_cloud)

            for gateway in range(0, gateways_per_cloud):

                current_gateway_id = self.next_id()
                new_gateway = Gateway(current_gateway_id, None, 0, current_cloud_id, self)

                # self.grid._place_agent((0, gateway ), new_gateway)
                self.schedule.add(new_gateway)

                for sensor in range(0, sensors_per_gateway):

                    current_sensor_id = self.next_id()
                    pos = (random.randint(0,width -1), random.randint(0,height -1))

                    new_sensor = Sensor( current_sensor_id, pos, 10, current_gateway_id, 
                                        1, sensor_accuracy, sensor_precision, self)

                    # self.grid._place_agent(pos, new_sensor)
                    self.schedule.add(new_sensor)




    def step(self):
        print("STEP:", self.schedule.steps)
        self.schedule.step()
        self.datacollector.collect(self)

        # if self.duration <

    def show_agents(self):
        print(self._agents)
    
    def calc_actual(self, pos):
        # Here we implement some function to simulate environmental 
        # variability across space and over time
        
        # but for now,
        return 10.0
    
    @staticmethod
    def retrieve_state_updates(model):
        state_updates = {}
        # # print(model.schedule.agents)
        # for breed in model.schedule.agents_by_breed:
        #     print(breed)
        #     state_updates[breed] = {}
        #     for agent in model.schedule.agents_by_breed[breed]:
        #         state_updates[breed][agent] = model.schedule.agents_by_breed[breed][agent].state_updates
        #     # recordings[sensor.pos] = sensor.recordings

        return state_updates

In [189]:
# unique_id, pos, sync_freq, parent_gateway, accuracy, precision, model


m = CentralizedArchitecture(1,2,3, 0, 0.1)

In [190]:
for i in range(0,10):
    m.step()

STEP: 0
Actual: 10.0
Observed: 10.089230873523254
Actual: 10.0
Observed: 9.892506801360938
Actual: 10.0
Observed: 10.15813759003923
Actual: 10.0
Observed: 10.000640912400222
Actual: 10.0
Observed: 9.940610153251594
Actual: 10.0
Observed: 10.01674290142972
STEP: 1
Actual: 10.0
Observed: 9.952878345498462
Actual: 10.0
Observed: 10.057846107618165
Actual: 10.0
Observed: 10.162122987235275
Actual: 10.0
Observed: 9.72536779601597
Actual: 10.0
Observed: 9.699206817238966
Actual: 10.0
Observed: 9.914407847897705
STEP: 2
Actual: 10.0
Observed: 10.052439154702984
Actual: 10.0
Observed: 10.024882552477061
Actual: 10.0
Observed: 10.137341589902618
Actual: 10.0
Observed: 10.010006783728175
Actual: 10.0
Observed: 9.931129533657236
Actual: 10.0
Observed: 10.015079668407031
STEP: 3
Actual: 10.0
Observed: 10.030408842223556
Actual: 10.0
Observed: 9.869035679339122
Actual: 10.0
Observed: 10.159088393492796
Actual: 10.0
Observed: 10.001781884270372
Actual: 10.0
Observed: 9.860537908019227
Actual: 10.0
O

In [191]:
m.schedule._agents[9].summarize(2,5)

,actual,observed
count,3.0,3.000000
mean,10.0,9.931265
std,0.0,0.062297
min,10.0,9.869036
25%,10.0,9.900083
50%,10.0,9.931130
75%,10.0,9.962379
max,10.0,9.993629
start_tick,2.0,2.000000
end_tick,5.0,5.000000


In [161]:
m.schedule._agents[9].records.dtypes

actual      float64
observed    float64
dtype: object